In [1]:
from PIL import Image, ImageEnhance, ImageFilter  
import os
from glob import glob

import pandas as pd
import numpy as np
import random

In [2]:
def create_folder(path, folder = ""):
    
    final_path = os.path.join(path, folder)
    
    if not os.path.isdir(final_path):
        os.makedirs (final_path)
    
    return final_path

In [3]:
def images_count(path_folder):
    lst = os.listdir(path_folder) 
    count = len(lst)
    return count

In [4]:
# https://www.codementor.io/@isaib.cicourel/image-manipulation-in-python-du1089j1u

# Return color value depending on quadrant and saturation
def get_saturation(value, quadrant):
  if value > 223:
    return 255
  elif value > 159:
    if quadrant != 1:
      return 255

    return 0
  elif value > 95:
    if quadrant == 0 or quadrant == 3:
      return 255

    return 0
  elif value > 32:
    if quadrant == 1:
      return 255

    return 0
  else:
    return 0

# Get the pixel from the given image
def get_pixel(image, i, j):
  # Inside image bounds?
  width, height = image.size
  if i > width or j > height:
    return None

  # Get Pixel
  pixel = image.getpixel((i, j))
  return pixel

# Create a dithered version of the image
def convert_dithering(image):
    # Get size
    width, height = image.size

    # Create new Image and a Pixel Map
    new = create_image_white(width, height)
    pixels = new.load()

    # Transform to half tones
    for i in range(0, width, 2):
        for j in range(0, height, 2):
          # Get Pixels
          p1 = get_pixel(image, i, j)
          p2 = get_pixel(image, i, j + 1)
          p3 = get_pixel(image, i + 1, j)
          p4 = get_pixel(image, i + 1, j + 1)

          # Color Saturation by RGB channel
          red   = (p1[0] + p2[0] + p3[0] + p4[0]) / 4
          green = (p1[1] + p2[1] + p3[1] + p4[1]) / 4
          blue  = (p1[2] + p2[2] + p3[2] + p4[2]) / 4

          # Results by channel
          r = [0, 0, 0, 0]
          g = [0, 0, 0, 0]
          b = [0, 0, 0, 0]

          # Get Quadrant Color
          for x in range(0, 4):
            r[x] = get_saturation(red, x)
            g[x] = get_saturation(green, x)
            b[x] = get_saturation(blue, x)

          # Set Dithered Colors
          pixels[i, j]         = (r[0], g[0], b[0])
          pixels[i, j + 1]     = (r[1], g[1], b[1])
          pixels[i + 1, j]     = (r[2], g[2], b[2])
          pixels[i + 1, j + 1] = (r[3], g[3], b[3])

    # Return new image
    return new

In [5]:
def convert_brightness(image, brightness):
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(brightness)
    

In [6]:
def image_name_brightness(name, brightness):
    return ''.join([name, "_", f"{brightness}".replace('.', '_')])

In [7]:
def image_name_dithering(name):
    return ''.join([name, "_dither"])

In [8]:
def image_name_low_quality(name):
    return ''.join([name, "_low"])

In [9]:
def save_image(img, path, name, fillcolor,  quality=75, extension='png'):

    
    img = img.rotate(random.randint(5, 175), Image.NEAREST, expand = False, fillcolor = fillcolor)
    img.save(f"{path}/{name}.{extension}", quality=quality)

In [10]:
# Create a new image with the given size
def create_image_white(i, j):
  image = Image.new("RGB", (i, j), "white")
  return image

In [11]:
def create_image(path, name, r, g, b, override = False, width = 128, height = 128, brightness = 1, extension='png'):
    print(path, name, r, g, b, brightness)
    create_folder(path)
    
    name_brightness = image_name_brightness(name, brightness)
    name_dithering = image_name_dithering(name_brightness)
    name_low_quality = image_name_low_quality(name_brightness)
    
    if glob(f"{path}/{image_name_brightness}.{extension}"):
        #print("already exists")
        if override:
            pass
            
    else:
        #print("let's go !")
        img  = Image.new( mode = "RGB", size = (width, height), color = (r, g, b) )
        
        img = convert_brightness(img, brightness)
        save_image(img, path, name_brightness, (r, g, b))
        
         
        save_image(img, path, name_low_quality, (r, g, b), 1)
        
        # Baisse les perfs
        #img = convert_dithering(img)
        #save_image(img, path, name_dithering, (r, g, b))
        
        
    

In [12]:

df = pd.read_csv("./dataset/html_color_names.csv", sep=";")

df.head()

,color,enable,color_name,hex_code,r,g,b,remark
0,red,1,IndianRed,#CD5C5C,205,92,92,NaN
1,red,1,LightCoral,#F08080,240,128,128,NaN
2,red,1,Salmon,#FA8072,250,128,114,NaN
3,red,0,DarkSalmon,#E9967A,233,150,122,NaN
4,red,0,LightSalmon,#FFA07A,255,160,122,NaN


In [13]:
images_folder = "./dataset/images/"

In [14]:

for index, row in df.iterrows():
    for i in range(9, 14):
        if row['enable'] == 1:
            create_image(
                f"{images_folder}{row['color']}", 
                row['color_name'], 
                r = row['r'], 
                g = row['g'], 
                b = row['b'], 
                brightness=i/10)
    

./dataset/images/red IndianRed 205 92 92 0.9
./dataset/images/red IndianRed 205 92 92 1.0
./dataset/images/red IndianRed 205 92 92 1.1
./dataset/images/red IndianRed 205 92 92 1.2
./dataset/images/red IndianRed 205 92 92 1.3
./dataset/images/red LightCoral 240 128 128 0.9
./dataset/images/red LightCoral 240 128 128 1.0
./dataset/images/red LightCoral 240 128 128 1.1
./dataset/images/red LightCoral 240 128 128 1.2
./dataset/images/red LightCoral 240 128 128 1.3
./dataset/images/red Salmon 250 128 114 0.9
./dataset/images/red Salmon 250 128 114 1.0
./dataset/images/red Salmon 250 128 114 1.1
./dataset/images/red Salmon 250 128 114 1.2
./dataset/images/red Salmon 250 128 114 1.3
./dataset/images/red Crimson 220 20 60 0.9
./dataset/images/red Crimson 220 20 60 1.0
./dataset/images/red Crimson 220 20 60 1.1
./dataset/images/red Crimson 220 20 60 1.2
./dataset/images/red Crimson 220 20 60 1.3
./dataset/images/red Red 255 0 0 0.9
./dataset/images/red Red 255 0 0 1.0
./dataset/images/red Red 2

./dataset/images/purple DarkMagenta 139 0 139 1.1
./dataset/images/purple DarkMagenta 139 0 139 1.2
./dataset/images/purple DarkMagenta 139 0 139 1.3
./dataset/images/purple Purple 128 0 128 0.9
./dataset/images/purple Purple 128 0 128 1.0
./dataset/images/purple Purple 128 0 128 1.1
./dataset/images/purple Purple 128 0 128 1.2
./dataset/images/purple Purple 128 0 128 1.3
./dataset/images/purple Indigo 75 0 130 0.9
./dataset/images/purple Indigo 75 0 130 1.0
./dataset/images/purple Indigo 75 0 130 1.1
./dataset/images/purple Indigo 75 0 130 1.2
./dataset/images/purple Indigo 75 0 130 1.3
./dataset/images/purple SlateBlue 106 90 205 0.9
./dataset/images/purple SlateBlue 106 90 205 1.0
./dataset/images/purple SlateBlue 106 90 205 1.1
./dataset/images/purple SlateBlue 106 90 205 1.2
./dataset/images/purple SlateBlue 106 90 205 1.3
./dataset/images/purple DarkSlateBlue 72 61 139 0.9
./dataset/images/purple DarkSlateBlue 72 61 139 1.0
./dataset/images/purple DarkSlateBlue 72 61 139 1.1
./da

./dataset/images/brown Chocolate 210 105 30 1.3
./dataset/images/brown SaddleBrown 139 69 19 0.9
./dataset/images/brown SaddleBrown 139 69 19 1.0
./dataset/images/brown SaddleBrown 139 69 19 1.1
./dataset/images/brown SaddleBrown 139 69 19 1.2
./dataset/images/brown SaddleBrown 139 69 19 1.3
./dataset/images/brown Brown 165 42 42 0.9
./dataset/images/brown Brown 165 42 42 1.0
./dataset/images/brown Brown 165 42 42 1.1
./dataset/images/brown Brown 165 42 42 1.2
./dataset/images/brown Brown 165 42 42 1.3
./dataset/images/brown Maroon 128 0 0 0.9
./dataset/images/brown Maroon 128 0 0 1.0
./dataset/images/brown Maroon 128 0 0 1.1
./dataset/images/brown Maroon 128 0 0 1.2
./dataset/images/brown Maroon 128 0 0 1.3
./dataset/images/white White 255 255 255 0.9
./dataset/images/white White 255 255 255 1.0
./dataset/images/white White 255 255 255 1.1
./dataset/images/white White 255 255 255 1.2
./dataset/images/white White 255 255 255 1.3
./dataset/images/white Snow 255 250 250 0.9
./dataset/ima

In [17]:
#files_count(images_folder+'yellow')

max_images_count = 2000

for c in next(os.walk(images_folder))[1]:
    full_path = images_folder+c #images_folder+c
    img_count = images_count(full_path) # files_color_count = 
    print(c, img_count)
    
    for i in range(img_count, max_images_count):
        image_to_copy = os.listdir(full_path)[random.randint(0, img_count-1)]
        image_name_to_copy, image_extension = os.path.splitext(image_to_copy)
        
        img_original = Image.open(full_path+'/'+image_to_copy)
        width, height = img_original.size
        
        colors = img_original.getpixel((1, 1)) #((random.randint(0, width-1), random.randint(0, height-1)))
        r = colors[0]
        g = colors[1]
        b = colors[2]
        

        create_image(
            full_path, 
            image_name_to_copy+'_'+str(i), 
            r = r, 
            g = g, 
            b = b, 
            brightness=random.randint(9, 14)/10)        
        
        
        print(c, image_to_copy, image_name_to_copy, image_extension, r, g, b)
        
        if images_count(full_path) >= max_images_count:
            break

'''
    print(c, files_count(images_folder+'yellow')) # 
    

    files_count = files_count(images_folder+c)
    
    for i in range(files_count(images_folder + c), 1200):
        print(c, os.listdir(''.join([images_folder, c]))[1])
'''

blue 2000
brown 1980
./dataset/images/brown Brown_1_0_low_52_1_1_low_82_1_1_104_1_1_low_1980 165 42 42 1.4
brown Brown_1_0_low_52_1_1_low_82_1_1_104_1_1_low.png Brown_1_0_low_52_1_1_low_82_1_1_104_1_1_low .png 165 42 42
./dataset/images/brown Brown_0_9_122_1_0_130_1_3_219_1_4_275_1_0_303_0_9_304_1_2_314_0_9_340_1_4_395_1_0_425_1_2_431_1_0_469_0_9_481_1_1_489_1_1_510_1_0_520_0_9_low_540_1_3_553_1_4_572_1_1_594_1_4_616_0_9_617_1_1_625_1_0_low_635_1_4_651_0_9_low_1473_1_0_low_1981 165 42 42 0.9
brown Brown_0_9_122_1_0_130_1_3_219_1_4_275_1_0_303_0_9_304_1_2_314_0_9_340_1_4_395_1_0_425_1_2_431_1_0_469_0_9_481_1_1_489_1_1_510_1_0_520_0_9_low_540_1_3_553_1_4_572_1_1_594_1_4_616_0_9_617_1_1_625_1_0_low_635_1_4_651_0_9_low_1473_1_0_low.png Brown_0_9_122_1_0_130_1_3_219_1_4_275_1_0_303_0_9_304_1_2_314_0_9_340_1_4_395_1_0_425_1_2_431_1_0_469_0_9_481_1_1_489_1_1_510_1_0_520_0_9_low_540_1_3_553_1_4_572_1_1_594_1_4_616_0_9_617_1_1_625_1_0_low_635_1_4_651_0_9_low_1473_1_0_low .png 165 42 42
./datase

"\n    print(c, files_count(images_folder+'yellow')) # \n    \n\n    files_count = files_count(images_folder+c)\n    \n    for i in range(files_count(images_folder + c), 1200):\n        print(c, os.listdir(''.join([images_folder, c]))[1])\n"